In [1]:
import openai

In [2]:
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
import os

In [7]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "user", "content": "is it too late to join the course?"}]
)

In [10]:
response.choices[0].message.content

'It would depend on the specific course and institution offering it. Some courses may have a set enrollment period and may not allow new students to join after a certain date. It is best to contact the course provider or institution directly to inquire about their enrollment policies and see if it is still possible to join the course at this time.'